# Webscraping
Récupération des données portant sur les matchs de NBA entre la saison 2005-2006 et la saison 2018-2019.
Données en libre accès sur le site Basketball Reference.

URL : https://www.basketball-reference.com/leagues/

In [24]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [126]:
# Imports
import urllib
import bs4
import pandas as pd
from urllib import request

## Récupération des informations sur les matchs

In [127]:
list_years=[2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]
list_months=['october','november','december','january','february','march','april','may']
dict_months=dictionary = dict(zip(list_months, [10,11,12,1,2,3,4,5]))
colonnes = ['Date', 'Start (ET)', 'Visitor/Neutral', 'PTS', 'Home/Neutral', 'PTS', 
            'BoxScore', 'OT', 'Attend.', 'Notes']
list_teams=['Atlanta Hawks', 'Boston Celtics', 'Brooklyn Nets', 'Charlotte Hornets',
            'Chicago Bulls', 'Cleveland Cavaliers', 'Dallas Mavericks', 'Denver Nuggets', 
            'Detroit Pistons', 'Golden State Warriors', 'Houston Rockets', 'Indiana Pacers', 
            'LA Clippers', 'Los Angeles Lakers', 'Memphis Grizzlies', 'Miami Heat', 
            'Milwaukee Bucks', 'Minnesota Timberwolves', 'New Orleans Pelicans', 
            'New York Knicks', 'Oklahoma City Thunder', 'Orlando Magic', 
            'Philadelphia 76ers', 'Phoenix Suns', 'Portland Trail Blazers', 
            'Sacramento Kings', 'San Antonio Spurs', 'Toronto Raptors', 'Utah Jazz', 
            'Washington Wizards', 'New Jersey Nets', 'Charlotte Bobcats', 
            'New Orleans/Oklahoma City Hornets', 'New Orleans Hornets', 'Seattle SuperSonics', 
            'Los Angeles Clippers']
list_teams=['Atlanta Hawks', 'Boston Celtics', 'Brooklyn Nets', 'Charlotte Hornets',
            'Chicago Bulls', 'Cleveland Cavaliers', 'Dallas Mavericks', 'Denver Nuggets', 
            'Detroit Pistons', 'Golden State Warriors', 'Houston Rockets', 'Indiana Pacers', 
            'LA Clippers', 'Los Angeles Lakers', 'Memphis Grizzlies', 'Miami Heat', 
            'Milwaukee Bucks', 'Minnesota Timberwolves', 'New Orleans Pelicans', 
            'New York Knicks', 'Oklahoma City Thunder', 'Orlando Magic', 
            'Philadelphia 76ers', 'Phoenix Suns', 'Portland Trail Blazers', 
            'Sacramento Kings', 'San Antonio Spurs', 'Toronto Raptors', 'Utah Jazz', 
            'Washington Wizards']

In [128]:
def scrap_matchs(year,month):
    url = "https://www.basketball-reference.com/leagues/NBA_" + str(year) + "_games-" + str(month) + ".html"
    #print(url)
    try:
        request_text = request.urlopen(url).read()
        #print(request_text[:1000]) 
        page = bs4.BeautifulSoup(request_text, "lxml")
        tableau = page.find('table', {'class' : 'suppress_glossary sortable stats_table'})
        return tableau
    except request.HTTPError:
        return False

def determine_date(year,month):
    if month in ['october','november','december']:
        date=str([year-1,dict_months[month]])
    else:
        date=str([year,dict_months[month]])
    return date

def rename_teams(teams):
    if 'New Jersey Nets' in teams:
        teams = list(map(lambda x: x.replace('New Jersey Nets', 'Brooklyn Nets'), teams))
    if 'Charlotte Bobcats' in teams:
        teams = list(map(lambda x: x.replace('Charlotte Bobcats', 'Charlotte Hornets'), teams))
    if 'New Orleans/Oklahoma City Hornets' in teams:
        teams = list(map(lambda x: x.replace('New Orleans/Oklahoma City Hornets', 'New Orleans Pelicans'), teams))
    if 'New Orleans Hornets' in teams:
        teams = list(map(lambda x: x.replace('New Orleans Hornets', 'New Orleans Pelicans'), teams))
    if 'Seattle SuperSonics' in teams:
        teams = list(map(lambda x: x.replace('Seattle SuperSonics', 'Oklahoma City Thunder'), teams))
    if 'Los Angeles Clippers' in teams:
        teams = list(map(lambda x: x.replace('Los Angeles Clippers', 'LA Clippers'), teams))
    for team in teams:
        if team not in list_teams:
            print('----ERREUR----')
            print('Team :', team)
    return teams

In [134]:
df_final=pd.DataFrame(columns=colonnes+['Y-M'])
for year in list_years:
    playoffs=0
    for month in list_months:
        print(str(year)+ ' - ' + str(month))
        tableau=scrap_matchs(year,month)
        if tableau != False:
            date=determine_date(year,month)
            df_inter=pd.DataFrame(columns=colonnes)
            table_body = tableau.find('tbody')
            rows = table_body.find_all('tr')
            for row in rows:
                ligne=[row.find('th').text.strip()]
                cols = row.find_all('td')
                #col_1 = row.find('th').text.strip()
                #print(ligne)
                for ele in cols : 
                    ligne.append((ele.text).strip())
                print(ligne)
                if ligne != ['Playoffs'] and playoffs==0:
                    df_inter.loc[len(df_inter.index)] = ligne
                elif ligne == ['Playoffs']:
                    playoffs=1
            df_inter['Y-M']=date
            teams_1=rename_teams(df_inter['Visitor/Neutral'].tolist())
            df_inter['Visitor/Neutral']=teams_1
            teams_2=rename_teams(df_inter['Home/Neutral'].tolist())
            df_inter['Home/Neutral']=teams_2         
            df_final = pd.concat([df_final, df_inter], ignore_index=True)

2006 - october
2006 - november
['Tue, Nov 1, 2005', '7:00p', 'Milwaukee Bucks', '117', 'Philadelphia 76ers', '108', 'Box Score', 'OT', '17,705', '']
['Tue, Nov 1, 2005', '8:00p', 'Sacramento Kings', '67', 'New Orleans/Oklahoma City Hornets', '93', 'Box Score', '', '19,163', '']
['Tue, Nov 1, 2005', '8:00p', 'Denver Nuggets', '91', 'San Antonio Spurs', '102', 'Box Score', '', '18,797', '']
['Tue, Nov 1, 2005', '10:30p', 'Dallas Mavericks', '111', 'Phoenix Suns', '108', 'Box Score', '2OT', '17,961', '']
['Wed, Nov 2, 2005', '7:00p', 'New Orleans/Oklahoma City Hornets', '87', 'Cleveland Cavaliers', '109', 'Box Score', '', '20,562', '']
['Wed, Nov 2, 2005', '7:00p', 'Indiana Pacers', '90', 'Orlando Magic', '78', 'Box Score', '', '17,283', '']
['Wed, Nov 2, 2005', '7:00p', 'Washington Wizards', '99', 'Toronto Raptors', '96', 'Box Score', '', '18,106', '']
['Wed, Nov 2, 2005', '7:30p', 'New York Knicks', '100', 'Boston Celtics', '114', 'Box Score', 'OT', '18,624', '']
['Wed, Nov 2, 2005', '7

['Sun, Nov 20, 2005', '9:00p', 'Memphis Grizzlies', '83', 'Denver Nuggets', '99', 'Box Score', '', '16,072', '']
['Sun, Nov 20, 2005', '9:00p', 'Sacramento Kings', '104', 'Seattle SuperSonics', '106', 'Box Score', '', '16,113', '']
['Sun, Nov 20, 2005', '9:30p', 'Chicago Bulls', '96', 'Los Angeles Lakers', '93', 'Box Score', '', '18,997', '']
['Mon, Nov 21, 2005', '7:00p', 'New Orleans/Oklahoma City Hornets', '91', 'Philadelphia 76ers', '103', 'Box Score', '', '12,452', '']
['Mon, Nov 21, 2005', '9:00p', 'Milwaukee Bucks', '80', 'Utah Jazz', '100', 'Box Score', '', '16,176', '']
['Mon, Nov 21, 2005', '10:00p', 'San Antonio Spurs', '96', 'Sacramento Kings', '93', 'Box Score', '', '17,317', '']
['Mon, Nov 21, 2005', '10:30p', 'New Jersey Nets', '97', 'Golden State Warriors', '100', 'Box Score', '', '16,501', '']
['Tue, Nov 22, 2005', '7:00p', 'Boston Celtics', '93', 'Cleveland Cavaliers', '115', 'Box Score', '', '17,139', '']
['Tue, Nov 22, 2005', '7:00p', 'Denver Nuggets', '108', 'Washi

['Thu, Dec 1, 2005', '8:00p', 'San Antonio Spurs', '92', 'Dallas Mavericks', '90', 'Box Score', '', '20,076', '']
['Thu, Dec 1, 2005', '10:30p', 'Los Angeles Lakers', '105', 'Utah Jazz', '101', 'Box Score', 'OT', '18,008', '']
['Fri, Dec 2, 2005', '7:00p', 'Toronto Raptors', '102', 'Atlanta Hawks', '101', 'Box Score', '', '12,261', '']
['Fri, Dec 2, 2005', '7:00p', 'Milwaukee Bucks', '105', 'Washington Wizards', '102', 'Box Score', '', '20,173', '']
['Fri, Dec 2, 2005', '7:30p', 'Chicago Bulls', '106', 'Boston Celtics', '102', 'Box Score', '', '14,393', '']
['Fri, Dec 2, 2005', '8:00p', 'New York Knicks', '98', 'Detroit Pistons', '106', 'Box Score', '', '22,076', '']
['Fri, Dec 2, 2005', '8:00p', 'Orlando Magic', '69', 'Memphis Grizzlies', '91', 'Box Score', '', '13,913', '']
['Fri, Dec 2, 2005', '8:00p', 'Philadelphia 76ers', '86', 'New Orleans/Oklahoma City Hornets', '88', 'Box Score', '', '19,163', '']
['Fri, Dec 2, 2005', '9:00p', 'Denver Nuggets', '97', 'Phoenix Suns', '102', 'Box

['Fri, Dec 16, 2005', '10:30p', 'Washington Wizards', '91', 'Los Angeles Lakers', '97', 'Box Score', '', '18,997', '']
['Sat, Dec 17, 2005', '3:30p', 'Houston Rockets', '81', 'Los Angeles Clippers', '89', 'Box Score', '', '17,611', '']
['Sat, Dec 17, 2005', '7:30p', 'Detroit Pistons', '103', 'Charlotte Bobcats', '78', 'Box Score', '', '19,026', '']
['Sat, Dec 17, 2005', '7:30p', 'Miami Heat', '107', 'Cleveland Cavaliers', '115', 'Box Score', '', '20,562', '']
['Sat, Dec 17, 2005', '7:30p', 'Indiana Pacers', '102', 'New York Knicks', '96', 'Box Score', '', '19,763', '']
['Sat, Dec 17, 2005', '8:00p', 'Phoenix Suns', '87', 'Memphis Grizzlies', '91', 'Box Score', '', '16,375', '']
['Sat, Dec 17, 2005', '8:30p', 'Boston Celtics', '86', 'Chicago Bulls', '118', 'Box Score', '', '20,746', '']
['Sat, Dec 17, 2005', '8:30p', 'Utah Jazz', '88', 'Milwaukee Bucks', '80', 'Box Score', '', '16,827', '']
['Sat, Dec 17, 2005', '8:30p', 'Sacramento Kings', '89', 'San Antonio Spurs', '90', 'Box Score', 

['Sun, Jan 1, 2006', '6:00p', 'Minnesota Timberwolves', '70', 'Miami Heat', '97', 'Box Score', '', '20,274', '']
['Sun, Jan 1, 2006', '9:00p', 'Los Angeles Clippers', '100', 'Portland Trail Blazers', '94', 'Box Score', '', '13,792', '']
['Sun, Jan 1, 2006', '9:30p', 'Utah Jazz', '98', 'Los Angeles Lakers', '94', 'Box Score', '', '18,310', '']
['Mon, Jan 2, 2006', '7:00p', 'Seattle SuperSonics', '96', 'Indiana Pacers', '115', 'Box Score', '', '14,073', '']
['Mon, Jan 2, 2006', '7:30p', 'Phoenix Suns', '133', 'New York Knicks', '140', 'Box Score', '3OT', '19,763', '']
['Mon, Jan 2, 2006', '8:00p', 'Charlotte Bobcats', '86', 'New Orleans/Oklahoma City Hornets', '103', 'Box Score', '', '17,683', '']
['Mon, Jan 2, 2006', '8:30p', 'Milwaukee Bucks', '93', 'Chicago Bulls', '92', 'Box Score', '', '20,968', '']
['Mon, Jan 2, 2006', '9:00p', 'Boston Celtics', '110', 'Denver Nuggets', '120', 'Box Score', '', '14,722', '']
['Tue, Jan 3, 2006', '7:00p', 'Toronto Raptors', '108', 'Atlanta Hawks', '9

...

['Mon, Feb 11, 2019', '8:00p', 'Dallas Mavericks', '104', 'Houston Rockets', '120', 'Box Score', '', '18,055', '']
['Mon, Feb 11, 2019', '8:00p', 'Los Angeles Clippers', '120', 'Minnesota Timberwolves', '130', 'Box Score', '', '13,782', '']
['Mon, Feb 11, 2019', '8:00p', 'Portland Trail Blazers', '111', 'Oklahoma City Thunder', '120', 'Box Score', '', '18,203', '']
['Mon, Feb 11, 2019', '9:00p', 'Miami Heat', '87', 'Denver Nuggets', '103', 'Box Score', '', '18,378', '']
['Tue, Feb 12, 2019', '7:30p', 'Los Angeles Lakers', '113', 'Atlanta Hawks', '117', 'Box Score', '', '16,824', '']
['Tue, Feb 12, 2019', '8:00p', 'San Antonio Spurs', '108', 'Memphis Grizzlies', '107', 'Box Score', '', '13,788', '']
['Tue, Feb 12, 2019', '8:00p', 'Orlando Magic', '118', 'New Orleans Pelicans', '88', 'Box Score', '', '15,733', '']
['Tue, Feb 12, 2019', '8:00p', 'Boston Celtics', '112', 'Philadelphia 76ers', '109', 'Box Score', '', '20,582', '']
['Tue, Feb 12, 2019', '10:30p', 'Utah Jazz', '108', 'Golden 

['Fri, Mar 1, 2019', '7:30p', 'Chicago Bulls', '168', 'Atlanta Hawks', '161', 'Box Score', '4OT', '15,267', '']
['Fri, Mar 1, 2019', '7:30p', 'Charlotte Hornets', '123', 'Brooklyn Nets', '112', 'Box Score', '', '15,578', '']
['Fri, Mar 1, 2019', '8:00p', 'Washington Wizards', '96', 'Boston Celtics', '107', 'Box Score', '', '18,624', '']
['Fri, Mar 1, 2019', '8:00p', 'Portland Trail Blazers', '117', 'Toronto Raptors', '119', 'Box Score', '', '19,800', '']
['Fri, Mar 1, 2019', '9:00p', 'New Orleans Pelicans', '130', 'Phoenix Suns', '116', 'Box Score', '', '14,123', '']
['Fri, Mar 1, 2019', '10:00p', 'Los Angeles Clippers', '116', 'Sacramento Kings', '109', 'Box Score', '', '17,583', '']
['Fri, Mar 1, 2019', '10:30p', 'Milwaukee Bucks', '131', 'Los Angeles Lakers', '120', 'Box Score', '', '18,997', '']
['Sat, Mar 2, 2019', '5:00p', 'Detroit Pistons', '129', 'Cleveland Cavaliers', '93', 'Box Score', '', '19,432', '']
['Sat, Mar 2, 2019', '7:00p', 'Orlando Magic', '117', 'Indiana Pacers', '

['Mon, Mar 18, 2019', '8:00p', 'Golden State Warriors', '105', 'San Antonio Spurs', '111', 'Box Score', '', '18,354', '']
['Mon, Mar 18, 2019', '8:30p', 'New Orleans Pelicans', '129', 'Dallas Mavericks', '125', 'Box Score', 'OT', '20,276', '']
['Mon, Mar 18, 2019', '10:00p', 'Chicago Bulls', '116', 'Phoenix Suns', '101', 'Box Score', '', '15,879', '']
['Mon, Mar 18, 2019', '10:30p', 'Indiana Pacers', '98', 'Portland Trail Blazers', '106', 'Box Score', '', '19,393', '']
['Tue, Mar 19, 2019', '7:00p', 'Philadelphia 76ers', '118', 'Charlotte Hornets', '114', 'Box Score', '', '16,411', '']
['Tue, Mar 19, 2019', '7:30p', 'Houston Rockets', '121', 'Atlanta Hawks', '105', 'Box Score', '', '16,293', '']
['Tue, Mar 19, 2019', '8:00p', 'Los Angeles Lakers', '101', 'Milwaukee Bucks', '115', 'Box Score', '', '17,879', '']
['Tue, Mar 19, 2019', '8:00p', 'Golden State Warriors', '117', 'Minnesota Timberwolves', '107', 'Box Score', '', '17,964', '']
['Tue, Mar 19, 2019', '10:00p', 'Brooklyn Nets', '1

['Mon, Apr 1, 2019', '7:00p', 'Detroit Pistons', '102', 'Indiana Pacers', '111', 'Box Score', '', '15,760', '']
['Mon, Apr 1, 2019', '7:30p', 'Miami Heat', '105', 'Boston Celtics', '110', 'Box Score', '', '18,624', '']
['Mon, Apr 1, 2019', '7:30p', 'Milwaukee Bucks', '131', 'Brooklyn Nets', '121', 'Box Score', '', '17,732', '']
['Mon, Apr 1, 2019', '7:30p', 'Chicago Bulls', '105', 'New York Knicks', '113', 'Box Score', '', '18,874', '']
['Mon, Apr 1, 2019', '7:30p', 'Orlando Magic', '109', 'Toronto Raptors', '121', 'Box Score', '', '19,800', '']
['Mon, Apr 1, 2019', '8:00p', 'Portland Trail Blazers', '132', 'Minnesota Timberwolves', '122', 'Box Score', '', '11,209', '']
['Mon, Apr 1, 2019', '8:30p', 'Philadelphia 76ers', '102', 'Dallas Mavericks', '122', 'Box Score', '', '19,645', '']
['Mon, Apr 1, 2019', '9:00p', 'Charlotte Hornets', '102', 'Utah Jazz', '111', 'Box Score', '', '18,306', '']
['Mon, Apr 1, 2019', '10:00p', 'Cleveland Cavaliers', '113', 'Phoenix Suns', '122', 'Box Score'

['Wed, May 1, 2019', '9:00p', 'Portland Trail Blazers', '97', 'Denver Nuggets', '90', 'Box Score', '', '19,520', '']
['Thu, May 2, 2019', '8:00p', 'Toronto Raptors', '95', 'Philadelphia 76ers', '116', 'Box Score', '', '20,658', '']
['Fri, May 3, 2019', '8:00p', 'Milwaukee Bucks', '123', 'Boston Celtics', '116', 'Box Score', '', '18,624', '']
['Fri, May 3, 2019', '10:30p', 'Denver Nuggets', '137', 'Portland Trail Blazers', '140', 'Box Score', '4OT', '20,193', '']
['Sat, May 4, 2019', '8:30p', 'Golden State Warriors', '121', 'Houston Rockets', '126', 'Box Score', 'OT', '18,169', '']
['Sun, May 5, 2019', '3:30p', 'Toronto Raptors', '101', 'Philadelphia 76ers', '96', 'Box Score', '', '20,639', '']
['Sun, May 5, 2019', '7:00p', 'Denver Nuggets', '116', 'Portland Trail Blazers', '112', 'Box Score', '', '20,146', '']
['Mon, May 6, 2019', '7:00p', 'Milwaukee Bucks', '113', 'Boston Celtics', '101', 'Box Score', '', '18,624', '']
['Mon, May 6, 2019', '9:30p', 'Golden State Warriors', '108', 'Hou

In [135]:
df_final

,Date,Start (ET),Visitor/Neutral,PTS,Home/Neutral,PTS,BoxScore,OT,Attend.,Notes,Y-M
0,"Tue, Nov 1, 2005",7:00p,Milwaukee Bucks,117,Philadelphia 76ers,108,Box Score,OT,"17,705",,"[2005, 11]"
1,"Tue, Nov 1, 2005",8:00p,Sacramento Kings,67,New Orleans Pelicans,93,Box Score,,"19,163",,"[2005, 11]"
2,"Tue, Nov 1, 2005",8:00p,Denver Nuggets,91,San Antonio Spurs,102,Box Score,,"18,797",,"[2005, 11]"
3,"Tue, Nov 1, 2005",10:30p,Dallas Mavericks,111,Phoenix Suns,108,Box Score,2OT,"17,961",,"[2005, 11]"
4,"Wed, Nov 2, 2005",7:00p,New Orleans Pelicans,87,Cleveland Cavaliers,109,Box Score,,"20,562",,"[2005, 11]"
...,...,...,...,...,...,...,...,...,...,...,...
16974,"Wed, Apr 10, 2019",8:00p,Chicago Bulls,109,Philadelphia 76ers,125,Box Score,,"20,197",,"[2019, 4]"
16975,"Wed, Apr 10, 2019",8:00p,Dallas Mavericks,94,San Antonio Spurs,105,Box Score,,"18,629",,"[2019, 4]"
16976,"Wed, Apr 10, 2019",10:30p,Minnesota Timberwolves,95,Denver Nuggets,99,Box Score,,"16,332",,"[2019, 4]"
16977,"Wed, Apr 10, 2019",10:30p,Utah Jazz,137,LA Clippers,143,Box Score,OT,"17,655",,"[2019, 4]"


In [136]:
df_final.to_csv('stats-matchs.csv', sep=';')

In [116]:
dof=pd.read_csv("Classeur1.csv", sep=';')

In [125]:
equipes=dof['Visitor/Neutral'].tolist()+dof['Home/Neutral'].tolist()
equipes=set(equipes)
equipes=list(equipes)
equipes.sort()
print(equipes)
k=0
for element in equipes:
    if element in list_teams:
        print("OK")
        k+=1
        print(k)
    else:
        print("NOOOOOON")

['Atlanta Hawks', 'Boston Celtics', 'Brooklyn Nets', 'Charlotte Hornets', 'Chicago Bulls', 'Cleveland Cavaliers', 'Dallas Mavericks', 'Denver Nuggets', 'Detroit Pistons', 'Golden State Warriors', 'Houston Rockets', 'Indiana Pacers', 'Los Angeles Clippers', 'Los Angeles Lakers', 'Memphis Grizzlies', 'Miami Heat', 'Milwaukee Bucks', 'Minnesota Timberwolves', 'New Orleans Pelicans', 'New York Knicks', 'Oklahoma City Thunder', 'Orlando Magic', 'Philadelphia 76ers', 'Phoenix Suns', 'Portland Trail Blazers', 'Sacramento Kings', 'San Antonio Spurs', 'Toronto Raptors', 'Utah Jazz', 'Washington Wizards']
OK
1
OK
2
OK
3
OK
4
OK
5
OK
6
OK
7
OK
8
OK
9
OK
10
OK
11
OK
12
NOOOOOON
OK
13
OK
14
OK
15
OK
16
OK
17
OK
18
OK
19
OK
20
OK
21
OK
22
OK
23
OK
24
OK
25
OK
26
OK
27
OK
28
OK
29


In [51]:
df_inter=pd.DataFrame(columns=colonnes)

In [30]:
# Trouver les noms des colonnes
table_head = tableau.find('thead')
#print(table_head)
row = table_head.find_all('th')
print(row)
cols = [ele.text.strip() for ele in row]
print(cols)


[<th aria-label="Date" class="poptip sort_default_asc center" data-stat="date_game" scope="col">Date</th>, <th aria-label="Start (ET)" class="poptip center" data-stat="game_start_time" scope="col">Start (ET)</th>, <th aria-label="Visitor/Neutral" class="poptip sort_default_asc center" data-stat="visitor_team_name" scope="col">Visitor/Neutral</th>, <th aria-label="Visitor Points" class="poptip center" data-stat="visitor_pts" data-tip="Points" scope="col">PTS</th>, <th aria-label="Home/Neutral" class="poptip sort_default_asc center" data-stat="home_team_name" scope="col">Home/Neutral</th>, <th aria-label="Home Points" class="poptip center" data-stat="home_pts" scope="col">PTS</th>, <th aria-label=" " class="poptip sort_default_asc center" data-stat="box_score_text" scope="col"> </th>, <th aria-label=" " class="poptip center" data-stat="overtimes" scope="col"> </th>, <th aria-label="Attend." class="poptip center" data-stat="attendance" scope="col">Attend.</th>, <th aria-label="Notes" clas

In [52]:
table_body = tableau.find('tbody')
table_body

<tbody><tr><th class="left" csk="200610310MIA" data-stat="date_game" scope="row"><a href="/boxscores/index.fcgi?month=10&amp;day=31&amp;year=2006">Tue, Oct 31, 2006</a></th><td class="right" data-stat="game_start_time">8:00p</td><td class="left" csk="CHI.200610310MIA" data-stat="visitor_team_name"><a href="/teams/CHI/2007.html">Chicago Bulls</a></td><td class="right" data-stat="visitor_pts">108</td><td class="left" csk="MIA.200610310MIA" data-stat="home_team_name"><a href="/teams/MIA/2007.html">Miami Heat</a></td><td class="right" data-stat="home_pts">66</td><td class="center" data-stat="box_score_text"><a href="/boxscores/200610310MIA.html">Box Score</a></td><td class="center iz" data-stat="overtimes"></td><td class="right" data-stat="attendance">20,229</td><td class="left iz" data-stat="game_remarks"></td></tr>
<tr><th class="left" csk="200610310LAL" data-stat="date_game" scope="row"><a href="/boxscores/index.fcgi?month=10&amp;day=31&amp;year=2006">Tue, Oct 31, 2006</a></th><td class

In [53]:
rows = table_body.find_all('tr')
print(rows[0])
print(rows[1])

<tr><th class="left" csk="200610310MIA" data-stat="date_game" scope="row"><a href="/boxscores/index.fcgi?month=10&amp;day=31&amp;year=2006">Tue, Oct 31, 2006</a></th><td class="right" data-stat="game_start_time">8:00p</td><td class="left" csk="CHI.200610310MIA" data-stat="visitor_team_name"><a href="/teams/CHI/2007.html">Chicago Bulls</a></td><td class="right" data-stat="visitor_pts">108</td><td class="left" csk="MIA.200610310MIA" data-stat="home_team_name"><a href="/teams/MIA/2007.html">Miami Heat</a></td><td class="right" data-stat="home_pts">66</td><td class="center" data-stat="box_score_text"><a href="/boxscores/200610310MIA.html">Box Score</a></td><td class="center iz" data-stat="overtimes"></td><td class="right" data-stat="attendance">20,229</td><td class="left iz" data-stat="game_remarks"></td></tr>
<tr><th class="left" csk="200610310LAL" data-stat="date_game" scope="row"><a href="/boxscores/index.fcgi?month=10&amp;day=31&amp;year=2006">Tue, Oct 31, 2006</a></th><td class="right

In [54]:
date='[2005,10]'
for row in rows:
    cols = row.find_all('td')
    ligne=[row.find('th').text.strip()]
    #col_1 = row.find('th').text.strip()
    #print(ligne)
    for ele in cols : 
        ligne.append((ele.text).strip())
    print(ligne)
    df_inter.loc[len(df_inter.index)] = ligne
df_inter['Y-M']=date

['Tue, Oct 31, 2006', '8:00p', 'Chicago Bulls', '108', 'Miami Heat', '66', 'Box Score', '', '20,229', '']
['Tue, Oct 31, 2006', '10:30p', 'Phoenix Suns', '106', 'Los Angeles Lakers', '114', 'Box Score', '', '18,997', '']


In [55]:
df_inter
df_final = pd.concat([df_final, df_inter], ignore_index=True)

,Date,Start (ET),Visitor/Neutral,PTS,Home/Neutral,PTS,Box Score,OT,Attend.,Notes,Y-M
0,"Tue, Oct 31, 2006",8:00p,Chicago Bulls,108,Miami Heat,66,Box Score,,"20,229",,"[2005,10]"
1,"Tue, Oct 31, 2006",10:30p,Phoenix Suns,106,Los Angeles Lakers,114,Box Score,,"18,997",,"[2005,10]"


In [36]:
import requests
html = request.urlopen(url_base_nba_players)
all = request_text.content
#print(request_text[:1000])    
soup = bs4.BeautifulSoup(html)
print(all)

b'<!DOCTYPE html>\n<html xmlns:ng="http://angularjs.org" lang="en" ng-app="stats" ng-strict-di stats-version="4.47.0" stats-build-date="2021-11-24 01:44 PM">\n\n  <head>\n    <meta charset="utf-8">\n    <meta http-equiv="X-UA-Compatible" content="IE=Edge,chrome=1"><script type="text/javascript">(window.NREUM||(NREUM={})).init={ajax:{deny_list:["bam-cell.nr-data.net"]}};(window.NREUM||(NREUM={})).loader_config={xpid:"VQECWF5UChAHUlNTBwgBVw==",licenseKey:"09f0cb5c68",applicationID:"76210961"};window.NREUM||(NREUM={}),__nr_require=function(t,e,n){function r(n){if(!e[n]){var i=e[n]={exports:{}};t[n][0].call(i.exports,function(e){var i=t[n][1][e];return r(i||e)},i,i.exports)}return e[n].exports}if("function"==typeof __nr_require)return __nr_require;for(var i=0;i<n.length;i++)r(n[i]);return r}({1:[function(t,e,n){function r(t){try{s.console&&console.log(t)}catch(e){}}var i,o=t("ee"),a=t(28),s={};try{i=localStorage.getItem("__nr_flags").split(","),console&&"function"==typeof console.log&&(s.c